# Inference time measurement 

To be able to run this experimentation, [Grid2op](https://github.com/rte-france/Grid2Op) (1.7.1) and [LightSim2Grid](https://github.com/BDonnot/lightsim2grid) (0.7.0) are required. See the corresponding links for more information concerning their installation.

In [1]:
import pathlib
import numpy as np
from pprint import pprint
from matplotlib import pyplot as plt
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

# TOC
- [Device Selection](#device)
- [Environment: `l2rpn_case14_sandbox`](#env_case14)
    - [Benchmark1](#benchmark1)
        - [Physics solver time](#14_solver_bench1)
    - [Benchmark2](#benchmark2)
        - [Physics solver time](#14_solver_bench2)
- [Environment: `l2rpn_neurips_2020_track1_small`](#env_nips)
    - [Benchmark1](#nips_benchmark1)
        - [Physics solver time](#nips_solver_bench1)
    - [Benchmark2](#nips_benchmark2)
        - [Physics solver time](#nips_solver_bench2)

## Select a device on which the model should be evaluated <a id="device"></a>
On the basis of GPU or CPU device, the results may be vary significantly. You can choose one of the following block in each kernel to using whether GPU or CPU device. 

### Using A GPU

In [2]:
import tensorflow as tf
device = "GPU"
gpus = tf.config.experimental.list_physical_devices(device)
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], device)
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

### Using CPU

In [2]:
import os
device = "CPU"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

### Show the list of used devices

In [3]:
import tensorflow as tf
tf.config.experimental.get_visible_devices()

2022-06-09 12:15:03.211276: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-09 12:15:03.211307: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: rte-01
2022-06-09 12:15:03.211312: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: rte-01
2022-06-09 12:15:03.211410: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.43.4
2022-06-09 12:15:03.211427: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.43.4
2022-06-09 12:15:03.211431: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.43.4


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

<h1 style="color:green">Environment : "l2rpn_case14_sandbox"</h1> <a id="env_case14"></a>

In [426]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
#BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
# DO NOT CONSIDER THE TAU VECTOR
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox_time.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

<h2 style="color:orange"> Benchmark1 </h2> <a id="benchmark1"></a>

In [427]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="14_solver_bench1"></a>

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{\#Samples}_\textrm{# situations} \times \underbrace{(3! + 1)}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{20}$$, where $20$ is number of the lines in this benchmark.

#### This step is added to consider preprocessing for solver

In [ ]:
NB_RUN = 3
data_size = 1000

time_physics_solver = list()
time_preproc = list()
for i in range(NB_RUN):
    benchmark1_tmp = PowerGridBenchmark(benchmark_name="Benchmark1",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
    
    benchmark1_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(data_size),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark1_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    time_preproc.append(simulator._timer_preproc)
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [337]:
nb_samples = int(1e6)
time_preprocess = (np.mean(time_preproc) / 1000) * 1e6
print(f"preprocessing time required for generating {nb_samples} samples: {time_preprocess:.2f}s")

preprocessing time required for generating 1000000 samples: 36.64s


#### Security Analysis
Do the Security Analysis. Source : https://github.com/BDonnot/lightsim2grid/blob/master/examples/security_analysis.py

In [338]:
import time
import grid2op
from grid2op.Parameters import Parameters
from lightsim2grid import LightSimBackend, SecurityAnalysis

env_name = "l2rpn_case14_sandbox"

test = False
# Create the grid2op environment
param = Parameters()
param.NO_OVERFLOW_DISCONNECTION = True
env = grid2op.make(env_name,
                   backend=LightSimBackend(),
                   # ignore the protection, that are NOT simulated
                   # by the TimeSerie module !
                   param=param,
                   test=test)

# Run the environment on a scenario using the TimeSerie module
NB_RUN = 10
total_times = list()
s1 = list()
s2 = list()
s3 = list()
for i in range(NB_RUN):
    env.fast_forward_chronics(2*24*(60/5))
    beg_ = time.perf_counter()
    security_analysis = SecurityAnalysis(env)
    s1.append(time.perf_counter() - beg_)
    beg_ = time.perf_counter()
    security_analysis.add_all_n1_contingencies()
    s2.append(time.perf_counter() - beg_)
    beg_ = time.perf_counter()
    p_or, a_or, voltages = security_analysis.get_flows()
    s3.append(time.perf_counter() - beg_)
    total_times.append(security_analysis.computer.total_time())
    
# the 3 lines above are the only lines you need to do to perform a security analysis !

computer = security_analysis.computer
print(f"For environment: {env_name} ({computer.nb_solved()} n-1 simulated)")
print(f"Total time spent in \"computer\" to solve everything: {1e3*computer.total_time():.1f}ms "
      f"({computer.nb_solved() / computer.total_time():.0f} pf / s), "
      f"{1000.*computer.total_time() / computer.nb_solved():.2f} ms / pf)")
print(f"\t - time to compute the coefficients to simulate line disconnection: {1e3*computer.preprocessing_time():.2f}ms")
print(f"\t - time to pre process Ybus: {1e3*computer.modif_Ybus_time():.2f}ms")
print(f"\t - time to perform powerflows: {1e3*computer.solver_time():.2f}ms "
      f"({computer.nb_solved() / computer.solver_time():.0f} pf / s, "
      f"{1000.*computer.solver_time() / computer.nb_solved():.2f} ms / pf)")
print(f"In addition, it took {1e3*computer.amps_computation_time():.2f} ms to retrieve the current "
      f"from the complex voltages (in total "
      f"{computer.nb_solved() / ( computer.total_time() + computer.amps_computation_time()):.1f} "
      "pf /s, "
      f"{1000.*( computer.total_time() + computer.amps_computation_time()) / computer.nb_solved():.2f} ms / pf)")

For environment: l2rpn_case14_sandbox (19 n-1 simulated)
Total time spent in "computer" to solve everything: 0.4ms (52512 pf / s), 0.02 ms / pf)
	 - time to compute the coefficients to simulate line disconnection: 0.02ms
	 - time to pre process Ybus: 0.02ms
	 - time to perform powerflows: 0.31ms (61632 pf / s, 0.02 ms / pf)
In addition, it took 0.01 ms to retrieve the current from the complex voltages (in total 50623.7 pf /s, 0.02 ms / pf)


In [339]:
nb_samples = int(1e6)
# Add the preprocessing time
# physics_solver_time = ((1e3*np.mean(total_times) / computer.nb_solved()) * nb_samples)/1e3 + time_preprocess
# Do not consider this preprocessing time
physics_solver_time = ((1e3*np.mean(total_times) / computer.nb_solved()) * nb_samples)/1e3
print(f"{physics_solver_time:.2f} s for {nb_samples} samples")

55.63 s for 1000000 samples


Security analysis init take more time than two other steps

In [340]:
np.mean(s1)

0.01222042003646493

In [341]:
np.mean(s2)

3.752887714654207e-05

In [342]:
np.mean(s3)

0.00043623053934425113

<h3 style="color:red"> Fully connected model </h3>

In [428]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="CONFIG2",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
tf_fc.train(train_dataset=benchmark1.train_dataset,
            val_dataset=benchmark1.val_dataset,
            epochs=2
           )

##### Augment the datasize to represent the realistic power network

In [430]:
def augment_data(benchmark, size=1e6):
    data_size = benchmark._test_dataset.size
    if size < data_size:
        raise ValueError("You cannot reduce the data size using this function")
    factor = int(size / data_size)
    
    for nm_, arr_ in benchmark._test_dataset.data.items():
        if nm_ == "line_status":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=int)
        elif nm_ == "topo_vect":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=bool)
        else:
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=np.float32)
        
    benchmark._test_dataset.size = int(size)
    return benchmark

In [431]:
benchmark1 = augment_data(benchmark1, size=nb_samples)

In [432]:
benchmark1._test_dataset.size

1000000

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=nb_samples,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

######  CPU

In [434]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.0008 ms/pf
Inference time from Machine Learning category and using CPU: 0.0008 ms/pf


In [435]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 0.83 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 0.76 s for 1000000 samples


Computing the speed up 

In [436]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 67.26
Speed up factor from ML point of view using FullyConnected model and CPU is: 73.20


<h3 style="color:red"> LeapNet model </h3>

In [370]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   topo_vect_to_tau="given_list",
                   kwargs_tau=[(4, (2, 1, 2, 1, 2)), (1, (1, 2, 1, 2, 2, 2)), (5, (1, 1, 2, 2, 1, 2, 2))],
                   log_path=LOG_PATH)

In [ ]:
leap_net.train(train_dataset=benchmark1.train_dataset,
               val_dataset=benchmark1.val_dataset,
               epochs=2
              )

##### Evaluate it multiple times

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    leapnet_metrics = benchmark1.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=nb_samples,
                                                    dataset="test",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["TIME_INF"])

In [357]:
leapnet_metrics

{'test': {'ML': {'MSE_avg': {'a_or': 3804.31103515625,
    'a_ex': 5778.8916015625},
   'MAE_avg': {'a_or': 31.03605079650879, 'a_ex': 40.569061279296875},
   'TIME_INF': 10.329449051991105},
  'Physics': {},
  'IndRed': {'TIME_INF': 9.30644133198075}}}

######  CPU

In [358]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / 10000) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.2f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.2f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.98 ms/pf
Inference time from Machine Learning category and using CPU: 1.00 ms/pf


In [359]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 9.76 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 10.04 s for 1000000 samples


Computing the speed up 

In [360]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 5.70
Speed up factor from ML point of view using FullyConnected model and CPU is: 5.54


<h2 style="color:orange"> Benchmark2</h2> <a id="benchmark2"></a>

In [373]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="14_solver_bench2"></a>

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
NB_RUN = 3
data_size = 1000
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
dc_time = list()
for i in range(NB_RUN):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(data_size),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark2_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    dc_time.append(simulator._timer_preproc)
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [376]:
physics_solver_time_env1_bench2 = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 over {} runs is: {}s for {} samples".format(NB_RUN, physics_solver_time, data_size))
print("Physics solver computation time for benchmark 2 over {} runs and for one powerflow is: {} ms/pf".format(NB_RUN, (physics_solver_time/data_size)*1000))

Physics solver computation time for benchmark 2 over 3 runs is: 55.627445337695505s for 1000 samples
Physics solver computation time for benchmark 2 over 3 runs and for one powerflow is: 55.627445337695505 ms/pf


In [377]:
nb_samples = int(1e6)
physics_solver_time = (physics_solver_time_env1_bench2/1000)*nb_samples
print(f"{physics_solver_time:.2f} s for {nb_samples} samples")

202.10 s for 1000000 samples


#### DC solver time corresponds to preprocessing where the solver is initialized using DC approximation

In [378]:
DC_solver_time = (np.mean(dc_time)/1000)*nb_samples
print(DC_solver_time)

36.12637775950134


##### Speed up for case14 network

In [280]:
0.02 / (((60/1e6)/20)*1000)

6.666666666666667

##### Speed up for neurips 2020 network

In [278]:
0.02 / (((61/1e6)/59)*1000)

19.34426229508197

<h3 style="color:red"> Fully Connected model</h3>

In [379]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark2",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="CONFIG2",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

train it

In [ ]:
tf_fc.train(train_dataset=benchmark2.train_dataset,
            val_dataset=benchmark2.val_dataset,
            epochs=2
           )

##### Augment the datasize to represent the realistic power network

In [382]:
def augment_data(benchmark, size=1e6):
    data_size = benchmark._test_dataset.size
    if size < data_size:
        raise ValueError("You cannot reduce the data size using this function")
    factor = int(size / data_size)
    
    for nm_, arr_ in benchmark._test_dataset.data.items():
        if nm_ == "line_status":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=int)
        elif nm_ == "topo_vect":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=bool)
        else:
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=np.float32)
        
    benchmark._test_dataset.size = int(size)
    return benchmark

In [383]:
benchmark2 = augment_data(benchmark2, size=nb_samples)

In [384]:
benchmark2._test_dataset.size

1000000

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=nb_samples,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [386]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.0061 ms/pf
Inference time from Machine Learning category and using CPU: 0.0011 ms/pf


In [387]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark2._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark2._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 6.11 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 1.06 s for 1000000 samples


Computing the speed up 

In [388]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 33.08
Speed up factor from ML point of view using FullyConnected model and CPU is: 190.43


<h3 style="color:red"> LeapNet model </h3>

In [ ]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    leapnet_metrics = benchmark2.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="all",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

<h1 style="color:green">Environment : "l2rpn_neurips_2020_track1_small"</h1> <a id="env_nips"></a>

In [396]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
#BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small.ini"
# DO NOT CONSIDER the TAU_VECTOR
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small_time.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

<h2 style="color:orange"> Benchmark1 </h2> <a id="nips_benchmark1"></a>

In [397]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="nips_solver_bench1"></a>

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{\# samples}_\textrm{# situations} \times \underbrace{(4! + 1)}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{58}$$, where $58$ is number of the lines for this environment.

#### Consider the preprocssing step for physical solver and security analysis

In [ ]:
NB_RUN = 3
data_size = 1000

time_physics_solver = list()
time_preproc = list()
for i in range(NB_RUN):
    benchmark1_tmp = PowerGridBenchmark(benchmark_name="Benchmark1",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
    
    benchmark1_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(data_size),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark1_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    time_preproc.append(simulator._timer_preproc)
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [395]:
nb_samples = int(1e6)
time_preprocess = (np.mean(time_preproc) / 1000) * 1e6
print(f"preprocessing time required for generating {nb_samples} samples: {time_preprocess:.2f}s")

preprocessing time required for generating 1000000 samples: 58.11s


#### Secrity analysis step

In [398]:
import grid2op
from grid2op.Parameters import Parameters
from lightsim2grid import LightSimBackend, SecurityAnalysis

env_name = "l2rpn_neurips_2020_track1_small"

test = False
# Create the grid2op environment
param = Parameters()
param.NO_OVERFLOW_DISCONNECTION = True
env = grid2op.make(env_name,
                   backend=LightSimBackend(),
                   # ignore the protection, that are NOT simulated
                   # by the TimeSerie module !
                   param=param,
                   test=test)

# Run the environment on a scenario using the TimeSerie module
NB_RUN = 10
total_times = list()
sa_time = list()
for i in range(NB_RUN):
    env.fast_forward_chronics(2*24*(60/5))
    beg_ = time.perf_counter()
    security_analysis = SecurityAnalysis(env)
    security_analysis.add_all_n1_contingencies()
    p_or, a_or, voltages = security_analysis.get_flows()
    sa_time.append(time.perf_counter() - beg_)
    total_times.append(security_analysis.computer.total_time())
    
# the 3 lines above are the only lines you need to do to perform a security analysis !

computer = security_analysis.computer
print(f"For environment: {env_name} ({computer.nb_solved()} n-1 simulated)")
print(f"Total time spent in \"computer\" to solve everything: {1e3*computer.total_time():.1f}ms "
      f"({computer.nb_solved() / computer.total_time():.0f} pf / s), "
      f"{1000.*computer.total_time() / computer.nb_solved():.2f} ms / pf)")
print(f"\t - time to compute the coefficients to simulate line disconnection: {1e3*computer.preprocessing_time():.2f}ms")
print(f"\t - time to pre process Ybus: {1e3*computer.modif_Ybus_time():.2f}ms")
print(f"\t - time to perform powerflows: {1e3*computer.solver_time():.2f}ms "
      f"({computer.nb_solved() / computer.solver_time():.0f} pf / s, "
      f"{1000.*computer.solver_time() / computer.nb_solved():.2f} ms / pf)")
print(f"In addition, it took {1e3*computer.amps_computation_time():.2f} ms to retrieve the current "
      f"from the complex voltages (in total "
      f"{computer.nb_solved() / ( computer.total_time() + computer.amps_computation_time()):.1f} "
      "pf /s, "
      f"{1000.*( computer.total_time() + computer.amps_computation_time()) / computer.nb_solved():.2f} ms / pf)")

/home/ubuntu/SYSTEMX/milad/venv/lips_milad/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_trafo.py:65: UserWarning: There were some Nan in the pp_net.trafo["tap_step_degree"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_degree\"], they have been replaced by 0")
/home/ubuntu/SYSTEMX/milad/venv/lips_milad/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_slack.py:113: UserWarning: We found either some slack coefficient to be < 0. or they were all 0.We set them all to 1.0 to avoid such issues
  warnings.warn("We found either some slack coefficient to be < 0. or they were all 0."


For environment: l2rpn_neurips_2020_track1_small (58 n-1 simulated)
Total time spent in "computer" to solve everything: 1.9ms (30569 pf / s), 0.03 ms / pf)
	 - time to compute the coefficients to simulate line disconnection: 0.01ms
	 - time to pre process Ybus: 0.13ms
	 - time to perform powerflows: 1.72ms (33637 pf / s, 0.03 ms / pf)
In addition, it took 0.09 ms to retrieve the current from the complex voltages (in total 29201.9 pf /s, 0.03 ms / pf)


In [399]:
nb_samples = int(1e6)
# with preprocessing step
# physics_solver_time = ((1e3*computer.total_time() / computer.nb_solved()) * nb_samples)/1e3 + time_preprocess
# without preprocessing step
physics_solver_time = ((1e3*computer.total_time() / computer.nb_solved()) * nb_samples)/1e3
print(f"{physics_solver_time:.2f} s for {nb_samples} samples")

90.82 s for 1000000 samples


In [238]:
np.mean(sa_time)

0.019630374596454205

<h3 style="color:red"> Fully connected model </h3>

In [400]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="CONFIG2",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
tf_fc.train(train_dataset=benchmark1.train_dataset,
            val_dataset=benchmark1.val_dataset,
            epochs=2
           )

##### Augment the datasize to represent the realistic power network

In [402]:
def augment_data(benchmark, size=1e6):
    data_size = benchmark._test_dataset.size
    if size < data_size:
        raise ValueError("You cannot reduce the data size using this function")
    factor = int(size / data_size)
    
    for nm_, arr_ in benchmark._test_dataset.data.items():
        if nm_ == "line_status":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=int)
        elif nm_ == "topo_vect":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=bool)
        else:
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=np.float32)
        
    benchmark._test_dataset.size = int(size)
    return benchmark

In [403]:
benchmark1 = augment_data(benchmark1, size=nb_samples)

In [404]:
benchmark1._test_dataset.size

1000000

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=nb_samples,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

######  CPU

In [406]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.0018 ms/pf
Inference time from Machine Learning category and using CPU: 0.0019 ms/pf


In [407]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 1.81 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 1.92 s for 1000000 samples


Computing the speed up 

In [408]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 50.24
Speed up factor from ML point of view using FullyConnected model and CPU is: 47.36


<h2 style="color:orange"> Benchmark2 </h2> <a id="nips_benchmark2"></a>

In [409]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="nips_solver_bench2"></a>

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
NB_RUN = 3
data_size = 1000
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
dc_time = list()
for i in range(NB_RUN):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(data_size),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark2_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    dc_time.append(simulator._timer_preproc)
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [411]:
physics_solver_time_env1_bench2 = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 over {} runs is: {}s for {} samples".format(NB_RUN, physics_solver_time, data_size))
print("Physics solver computation time for benchmark 2 over {} runs and for one powerflow is: {} ms/pf".format(NB_RUN, (physics_solver_time/data_size)*1000))

Physics solver computation time for benchmark 2 over 3 runs is: 90.81932193677179s for 1000 samples
Physics solver computation time for benchmark 2 over 3 runs and for one powerflow is: 90.81932193677179 ms/pf


In [412]:
nb_samples = int(1e6)
physics_solver_time = (physics_solver_time_env1_bench2/1000)*nb_samples
print(f"{physics_solver_time:.2f} s for {nb_samples} samples")

294.07 s for 1000000 samples


In [413]:
DC_solver_time = (np.mean(dc_time)/1000)*nb_samples
print(DC_solver_time)

58.00984528226157


In [283]:
283/61

4.639344262295082

<h3 style="color:red"> Fully Connected model </h3>

In [414]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="CONFIG2",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
tf_fc.train(train_dataset=benchmark2.train_dataset,
            val_dataset=benchmark2.val_dataset,
            epochs=2
           )

##### Augment the datasize to represent the realistic power network

In [416]:
def augment_data(benchmark, size=1e6):
    data_size = benchmark._test_dataset.size
    if size < data_size:
        raise ValueError("You cannot reduce the data size using this function")
    factor = int(size / data_size)
    
    for nm_, arr_ in benchmark._test_dataset.data.items():
        if nm_ == "line_status":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=int)
        elif nm_ == "topo_vect":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=bool)
        else:
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=np.float32)
        
    benchmark._test_dataset.size = int(size)
    return benchmark

In [417]:
benchmark2 = augment_data(benchmark2, size=nb_samples)

In [418]:
benchmark2._test_dataset.size

1000000

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=nb_samples,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [420]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.0060 ms/pf
Inference time from Machine Learning category and using CPU: 0.0034 ms/pf


In [421]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 6.01 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 3.44 s for 1000000 samples


Computing the speed up 

In [422]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 48.93
Speed up factor from ML point of view using FullyConnected model and CPU is: 85.55
